In [2]:
import rasterio
from osgeo import gdal
import numpy as np
from osgeo import osr


In [4]:
file_imgname = 'LAI_img_0615_clip_1002.tif'
imgpath = r'E:\Desktop\20220913-20220916-0919-0923\20220916-0921\Sklearn回归模型规整\训练数据\{}'.format(file_imgname)
src0 = gdal.Open(imgpath)
# 获取TIFF的范围信息
extend = src0.GetGeoTransform()
# 读取波段信息
img = src0.GetRasterBand(1).ReadAsArray()
img

array([[0.20455411, 0.19061646, 0.12152281, ..., 1.5441545 , 1.7427843 ,
        1.6314082 ],
       [0.24155286, 0.11996793, 0.09875754, ..., 1.8404377 , 1.5346193 ,
        0.51362425],
       [0.13464758, 0.05696604, 0.04889766, ..., 1.1394424 , 1.3415544 ,
        0.3928216 ],
       ...,
       [0.9130307 , 1.0376265 , 2.1219854 , ..., 0.3671901 , 0.28756535,
        0.43703735],
       [0.6525149 , 0.83737385, 1.4548136 , ..., 0.57414424, 0.3944538 ,
        0.67484957],
       [0.52307   , 1.1647736 , 1.3697717 , ..., 1.819569  , 1.1292155 ,
        0.6484961 ]], dtype=float32)

In [6]:
# 路径GEE
file_img1name = 'GEE_0615_clip_982.tif'
imgpath1 = r'E:\Desktop\20220913-20220916-0919-0923\20220916-0921\Sklearn回归模型规整\训练数据\{}'.format(file_img1name)
src1 = rasterio.open(imgpath1)
rat = src1.read(1)
rat

array([[559, 557, 555, ..., 692, 780, 780],
       [559, 557, 555, ..., 691, 780, 780],
       [559, 557, 555, ..., 690, 780, 780],
       ...,
       [444, 444, 418, ..., 515, 499, 493],
       [444, 444, 419, ..., 515, 499, 493],
       [444, 444, 419, ..., 515, 500, 493]], dtype=int16)

In [7]:
def xy_to_rowcol(extend, x, y):
    '''根据GDAL的六参数模型将给定的投影坐标转为影像图上坐标(行列号)
    根据GDAL的六参数模型将给定的投影或地理坐标转为影像图上坐标(行列号)
    :param extend: 图像的空间范围
    :param x:投影坐标x
    :param y:投影坐标y
    :return:投影坐标(x, y)对应的影像图上行列号(row, col)
'''
    a = np.array([[extend[1], extend[2]], [extend[4], extend[5]]])
    b = np.array([x - extend[0], y - extend[3]])
    row_col = np.linalg.solve(a, b)  # 使用numpy的linalg. solve进行二 元一次 方程的求解
    row = int(np.floor(row_col[1]))
    col = int(np.floor(row_col[0]))
    #
    return row, col

In [8]:
gdal.AllRegister()

filepath = imgpath1
dataset = gdal.Open(filepath)
adfGeoTransform = dataset.GetGeoTransform()

# 左上角地理坐标
print(adfGeoTransform[0])
print(adfGeoTransform[3])

nXSize = dataset.RasterXSize  # 列数
nYSize = dataset.RasterYSize  # 行数
print(nXSize)
print(nYSize)

602320.8531989538
3545574.9181152456
982
982


In [10]:
arrSlope = []  # 用于存储每个像素的（X，Y）坐标
raster = np.zeros((rat.shape[0], rat.shape[1]))
raster

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
for i in range(nYSize):
    row = []
    for j in range(nXSize):
        px = adfGeoTransform[0] + i * adfGeoTransform[1] + j * adfGeoTransform[2]
        py = adfGeoTransform[3] + i * adfGeoTransform[4] + j * adfGeoTransform[5]
        row, col = xy_to_rowcol(extend, px, py)
        raster[j, i] = img[row, col]
# 重采样LAI数据，命名LAI_img_0814_clip_982.tif
driver = gdal.GetDriverByName('GTiff')
dst_filename = 'LAI_img_0814_clip_982.tif'
dst_filepath = r'E:\Desktop\20220913-20220916-0919-0923\20220916-0921\Sklearn回归模型规整\重采样LAI数据\{}'.format(dst_filename)
dst_ds = driver.Create(dst_filepath, rat.shape[0], rat.shape[1], 1, gdal.GDT_Float32)

In [12]:
import pandas as pd
import time
import joblib
import gdal
from itertools import chain
from sklearn.model_selection import train_test_split
from sklearn import ensemble
import numpy as np

In [20]:
imgpath = r"./训练数据/LAI_img_0814_clip_982_V4.tif"
src0 = gdal.Open(imgpath)
# 获取TIFF的范围信息
extend = src0.GetGeoTransform()
# 读取波段信息
y_train = src0.GetRasterBand(1).ReadAsArray()
y_0 = list(chain.from_iterable(y_train))
datay = pd.DataFrame({'result': y_0})
Y = datay.loc[:, ('result')]
Y

0         0.175388
1         0.542404
2         0.447647
3         0.512051
4         0.363177
            ...   
964319    0.503337
964320    0.238824
964321    0.234159
964322    1.613273
964323    3.208965
Name: result, Length: 964324, dtype: float32

In [26]:
imgpath = r"./重采样LAI数据/LAI_img_0615_clip_982.tif"
src0 = gdal.Open(imgpath)
# 获取TIFF的范围信息
extend = src0.GetGeoTransform()
# 读取波段信息
y_train01 = src0.GetRasterBand(1).ReadAsArray()
y_1 = list(chain.from_iterable(y_train))
datay1 = pd.DataFrame(y_1)
Y = pd.concat([Y, datay1])
Y

,0
0,0.175388
1,0.542404
2,0.447647
3,0.512051
4,0.363177
...,...
964319,0.503337
964320,0.238824
964321,0.234159
964322,1.613273


In [27]:
imgpath = r"./训练数据/GEE_0814_clip_982.tif"
src1 = gdal.Open(imgpath)
# 获取TIFF的范围信息
extend_x = src1.GetGeoTransform()
# 读取波段信息
x0814_train_2 = src1.GetRasterBand(2).ReadAsArray()
x0814_train_3 = src1.GetRasterBand(3).ReadAsArray()
x0814_train_4 = src1.GetRasterBand(4).ReadAsArray()
x0814_train_8 = src1.GetRasterBand(8).ReadAsArray()
# 转为一维数组1,-1
x_2 = list(chain.from_iterable(x0814_train_2))
x_3 = list(chain.from_iterable(x0814_train_3))
x_4 = list(chain.from_iterable(x0814_train_4))
x_8 = list(chain.from_iterable(x0814_train_8))
data = pd.DataFrame({'bound2': x_2, 'bound3': x_3, 'bound4': x_4, 'bound8': x_8})
X = data.loc[:, ('bound2', 'bound3', 'bound4', 'bound8')]
X

,bound2,bound3,bound4,bound8
0,442,735,766,3409
1,233,473,315,3509
2,189,419,213,3608
3,199,432,228,3603
4,285,537,373,3478
...,...,...,...,...
964319,382,753,458,3422
964320,317,638,402,3613
964321,215,525,275,3696
964322,278,580,336,3659


In [2]:
imgpath = r"./训练数据/GEE_0814_clip_982.tif"  # 路径
# 打开文件 rasterio.io.DatasetReader,包含了栅格的所有信息
src1 = rasterio.open(imgpath)
rat = src1.read()

In [9]:
train = np.transpose(rat[1:9], (1, 2, 0))
train = train.reshape(-1, 8)
train

array([[ 769, 1200, 1192, ..., 2284, 2383, 2270],
       [ 669, 1084,  832, ..., 2326, 2279, 2265],
       [ 663, 1133,  782, ..., 2384, 2204, 2258],
       ...,
       [ 710, 1001, 1134, ..., 1958, 1959, 2138],
       [ 547,  819,  780, ..., 2744, 2690, 2893],
       [ 397,  670,  457, ..., 3068, 3579, 3203]], dtype=int16)

In [10]:
X = pd.DataFrame(train, columns=['B' + str(i) for i in range(1, 8)] + ['B8'])

,B1,B2,B3,B4,B5,B6,B7,B8
0,769,1200,1192,1784,2178,2284,2383,2270
1,669,1084,832,1762,2207,2326,2279,2265
2,663,1133,782,1732,2248,2384,2204,2258
3,647,1095,732,1732,2248,2384,2148,2258
4,671,1083,944,1711,1981,2205,2187,2143
...,...,...,...,...,...,...,...,...
964319,533,821,814,1252,1817,2056,1574,2095
964320,667,990,1089,1321,1747,1957,1809,2138
964321,710,1001,1134,1325,1748,1958,1959,2138
964322,547,819,780,1184,2303,2744,2690,2893


In [11]:
imgpath2 = r"./训练数据/GEE_0615_clip_982.tif"  # 路径
# 打开文件 rasterio.io.DatasetReader,包含了栅格的所有信息
src2 = rasterio.open(imgpath2)
rat2 = src2.read()
train2 = np.transpose(rat2[1:9], (1, 2, 0))
train2 = train.reshape(-1, 8)
df = pd.DataFrame(train2, columns=['B' + str(i) for i in range(1, 8)] + ['B8'])
df

,B1,B2,B3,B4,B5,B6,B7,B8
0,769,1200,1192,1784,2178,2284,2383,2270
1,669,1084,832,1762,2207,2326,2279,2265
2,663,1133,782,1732,2248,2384,2204,2258
3,647,1095,732,1732,2248,2384,2148,2258
4,671,1083,944,1711,1981,2205,2187,2143
...,...,...,...,...,...,...,...,...
964319,533,821,814,1252,1817,2056,1574,2095
964320,667,990,1089,1321,1747,1957,1809,2138
964321,710,1001,1134,1325,1748,1958,1959,2138
964322,547,819,780,1184,2303,2744,2690,2893


In [15]:
data = pd.concat([X, df])
data

,B1,B2,B3,B4,B5,B6,B7,B8
0,769,1200,1192,1784,2178,2284,2383,2270
1,669,1084,832,1762,2207,2326,2279,2265
2,663,1133,782,1732,2248,2384,2204,2258
3,647,1095,732,1732,2248,2384,2148,2258
4,671,1083,944,1711,1981,2205,2187,2143
...,...,...,...,...,...,...,...,...
964319,533,821,814,1252,1817,2056,1574,2095
964320,667,990,1089,1321,1747,1957,1809,2138
964321,710,1001,1134,1325,1748,1958,1959,2138
964322,547,819,780,1184,2303,2744,2690,2893


In [17]:

file_name = '2022-05-06_QJS2DL.tif'
name = file_name.split('.')
name[0]

'2022-05-06_QJS2DL'

In [20]:
import time

In [21]:
mdhms = time.strftime('%d%H%M', time.localtime(time.time()))
mdhms

'230910'

In [30]:
imgpath = r'E:\Desktop\20220913-20220916-0919-0923\20220916-0921\Sklearn回归模型规整\投影\2022-05-06_QJS2DL.tif'
src0 = gdal.Open(imgpath)
# 获取TIFF的范围信息
src0.GetProjection()
# srs.SetUTM(11, 1)

'PROJCS["WGS 84 / UTM zone 50N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",117],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32650"]]'

In [29]:
adfGeoTransform1 = (1076350.853198954, 10.0, 0.0, 3567604.9181152456, 0.0, -10.0)
adfGeoTransform1

(1076350.853198954, 10.0, 0.0, 3567604.9181152456, 0.0, -10.0)

In [27]:
576350.8531989538 + 500000

1076350.853198954

In [31]:
srs = osr.SpatialReference()
# srs.SetProjection(src0.GetProjection())
srs.SetUTM(117, 1)

6

In [32]:
srs.SetWellKnownGeogCS( 'NAD27' )

0

In [ ]:
srs.ExportToWkt()

In [10]:
import rasterio
from osgeo import gdal
import numpy as np
from osgeo import osr
imgpath = r"./重采样LAI数据/LAI_img_0615_clip_982.tif"
src1 = rasterio.open(imgpath)
rat = src1.read()
train = np.transpose(rat[1:9], (1, 2, 0))
# train = train.reshape(-1, 8)
train

array([], shape=(982, 982, 0), dtype=float32)

In [12]:
A = np.reshape(np.arange(24),(4,3,2)) ##生成一个数值从0到23，维度为（4,3,2）的数组
print(A)

[[[ 0  1]
  [ 2  3]
  [ 4  5]]

 [[ 6  7]
  [ 8  9]
  [10 11]]

 [[12 13]
  [14 15]
  [16 17]]

 [[18 19]
  [20 21]
  [22 23]]]


In [13]:
print('A的维度：',A.shape)

A的维度： (4, 3, 2)


In [19]:
B = np.reshape(A,(-1,12))
print(B)

[[ 0  1  2  3  4  5  6  7  8  9 10 11]
 [12 13 14 15 16 17 18 19 20 21 22 23]]


In [20]:
print('B的维度：',B.shape)

B的维度： (2, 12)
